TODO:
    - Fix case for titles
    - Create button for analytics
    - Create potential button to remove certain data
    - Cross data to find number of people linked to a company

In [1]:
import sys

sys.path.insert(1,r'C:\Users\juann\OneDrive\Documentos\GitHub\ETL-tools\src')
sys.path.insert(1, r'C:\Users\juann\OneDrive\Documentos\GitHub\Data-analysis\UK corporate - study case\src')

In [2]:
import pathlib
import polars as pl
import pandas as pd
import etl_tools
import etl_logger
import logging
import matplotlib.pyplot as plt
import re
import uk_viz
import plotly.graph_objects as go

In [3]:
import etl_tools
import pathlib
import etl_logger
import logging
import polars as pl
import pandas as pd
import datetime as dt
from datetime import datetime
import uk_wrangle
import re
import uk_viz

# Define paths
path = pathlib.Path(r'C:\Users\juann\OneDrive\Documentos\GitHub\Data-analysis\UK corporate - study case\data')
companies_data = path / 'companies.parquet'
officers_owners_data = path / 'officers_and_owners.parquet'

# Get logger
console = logging.StreamHandler()
logger = etl_logger.get_logger('logger', logging.WARNING, [console])

# Columns to exclude
comps_cols_excl = [
    'next_accounts_overdue', 'confirmation_statement_overdue', 'owners', 'officers',
    'average_number_employees_during_period', 'current_assets', 'last_accounts_period_end',
    'sic_codes', 'account_type', 'company_url'
]
officers_owners_cols_excl = ['company_country', 'person_id', 'person_url']

# Load data with polars
companies_lz = pl.scan_parquet(companies_data)
officers_owners_lz = pl.scan_parquet(officers_owners_data)

# Select relevant columns
companies_cols = [col for col in companies_lz.collect_schema().names() if col not in comps_cols_excl]
officers_owners_cols = [col for col in officers_owners_lz.collect_schema().names() if col not in officers_owners_cols_excl]

# Load dataframes
companies = etl_tools.read_parquet(logger, companies_data, cols=companies_cols).with_columns([
    pl.col('date_of_cessation').fill_null(pl.lit(dt.datetime.today().date())),
    pl.col('jurisdiction').fill_null('UK establishment')
]).to_pandas()

officers_owners = etl_tools.read_parquet(logger, officers_owners_data, cols=officers_owners_cols).to_pandas()
officers_owners = officers_owners.merge(companies, on='company_number', how='inner')

# Processing companies
companies = companies.assign(
    Year=lambda df: df['incorporation_date'].apply(uk_wrangle.get_year),
    city=lambda df: df['office_address'].apply(lambda address: uk_wrangle.process_address(address, logger)),
    num_days_active=lambda df: df.apply(lambda row: uk_wrangle.days_between_dates(row, logger), axis=1)
)

# Create year brackets
bins = [0, 360, 1800, 3600, 7200, float('inf')]
labels = ['<1', '1-5y', '5-10y', '10-20y', '>20y']
companies['Years bracket'] = pd.cut(companies['num_days_active'], bins=bins, labels=labels)

# Refine city and country extraction
countries = ['England', 'United Kingdom', 'Wales', 'Ireland', 'Scotland', 'Northern Ireland']
# companies.loc[companies['city'].isin(countries), 'city'] = companies.loc[companies['city'].isin(countries), 'office_address'].apply(
#     lambda address: uk_wrangle.process_country(address, logger)
# )

# Split active and inactive companies
active_companies = companies[companies['company_status'].isin(['Active', 'Open'])]
not_active_companies = companies[~companies['company_status'].isin(['Active', 'Open'])]

# Active Companies Charts
active_years = uk_viz.prepare_years_bracket_data(active_companies)


# Not Active Companies Charts
not_active_years = uk_viz.prepare_years_bracket_data(not_active_companies)

years_viz = uk_viz.create_toggleable_pie_charts([active_years, not_active_years],
                                          ['Years bracket', 'Years bracket'],
                                          ['size', 'size'],
                                          ['Active Companies by Years Bracket', 'Not Active Companies by Years Bracket'])

2025-01-07 17:21:54,714 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'RG9 1JW, Henley-On-Thames, S. Oxfordshire, 5 Whitamore Row': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:54,718 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'Birmingham, Poppleton & Appleby, The Silverworks 67-71 Northwood Street, Jewellery Quarter': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:54,732 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'W1U 7NA, 43-45 Dorset Street, London': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:54,734 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'NE8 4JR, Tyne & Wear, 264-266 Durham Road, Gateshead': cannot access local variable 'city_can

2025-01-07 17:21:54,803 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'W1T 6LQ, 29/30 Fitzroy Square, London': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:54,803 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'EC1V 1NW, 381-383 City Road, London': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:54,804 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'NW6 4NP, 4 Abbot's Place, London': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:54,805 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'W1T 3LJ, 14-15 Berners Street, London': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:54,805 - juann - logger - uk_w

2025-01-07 17:21:54,898 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'TD15 1DJ, Berwick-Upon-Tweed, Greaves West & Ayre, 17 Walker Gate': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:54,903 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'KT12 3JU, Walton-On-Thames, 30 Kenilworth Drive': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:54,904 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'DE1 2JJ, 1 & 2 Wilmot Street, Derby': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:54,905 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'CH64 9PA, 1-3 Chester Road, Neston': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01

2025-01-07 17:21:54,970 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'E1 5NF, 7-15 Greatorex Street, London': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:54,978 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'W4 1SY, 80-82 Chiswick High Road, London': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:54,979 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'ST4 8HD, Stoke-On-Trent, 31 Barlaston Old Road': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:54,980 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'ST6 6EU, Stoke-On-Trent, C/O Staffordshire Fine Ceramics, Williamson Street': cannot access local variable 'city_candidate' where it is not associated wit

2025-01-07 17:21:55,034 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'EC1A 7HU, 14-15 Newbury Street, London': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:55,044 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'WC2N 6AA, Adam House 7-10  Adam Street, London': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:55,045 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'SR4 7JA, Tyne & Wear, 191-193 Chester Road, Sunderland': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:21:55,047 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'LA11 6DW, Unit 8-9 Station Yard, Grange Over Sands': cannot access local variable 'city_candidate' where it is not associated with a value


...

2025-01-07 17:22:04,548 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'M2 4EQ, 57-63 Princess Street, Manchester': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,549 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'CV1 4AN, 42- 44 Hill Street, Coventry': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,551 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'Not Available': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,553 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'Not Available': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,554 - juann - logger - uk_wrangle.py @process_address #52 - ERRO

2025-01-07 17:22:04,571 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'Refer To Parent Registry, Branch Registration': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,571 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'Refer To Parent Registry, Branch Registration': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,571 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address '20253 Hamburg, Hoheluftchaussee 95': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,571 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'Bucks., 20/22 Queens Road, High Wycombe': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,572 -

2025-01-07 17:22:04,601 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address '30 Exchange St East, Liverpool 2': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,604 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'W1T 3LJ, 14-15 Berners Street, London': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,604 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'BT15 3GW, 35-39 York Road, Belfast': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,605 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'Branch Registration, Refer To Parent Registry': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,605 - juann - logger

2025-01-07 17:22:04,628 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'G12 0QF, 8-10 Balcarres Avenue, Glasgow': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,629 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'SE1 1EP, 5-7 Marshalsea Road Borough, London': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,630 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'LE1 3JN, 39/43 Bedford Street, South Leicester': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,630 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'WC2A 3LH, 66 Lincoln's Inn Fields, London': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:0

2025-01-07 17:22:04,650 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'Kirriemuir, Cortachy Home Farm': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,650 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address '15 Atholl Crescent, Edinburgh': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,650 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address 'Oldfield La., Greenford Middx., Icg Hse., Station Approach': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,652 - juann - logger - uk_wrangle.py @process_address #52 - ERROR - Error processing address '40/41 Whiskin Street, London Ec1r Obp': cannot access local variable 'city_candidate' where it is not associated with a value
2025-01-07 17:22:04,654 - juann - 

In [12]:
def create_toggleable_pie_charts(dataframes, labels_columns, values_columns, titles, width=800, height=600, category_orders=None):
    """
    Creates a Plotly figure with toggleable pie charts for multiple dataframes.

    Parameters:
        dataframes (list): List of DataFrames for pie charts.
        labels_columns (list): List of column names for labels in each DataFrame.
        values_columns (list): List of column names for values in each DataFrame.
        titles (list): List of titles for each pie chart.
        width (int): Width of the figure.
        height (int): Height of the figure.
        category_orders (list): List of dictionaries specifying the order of categories for each pie chart.
                                Each dictionary should map label column values to their desired order.

    Returns:
        Plotly Figure: The figure containing toggleable pie charts.
    """
    if not (len(dataframes) == len(labels_columns) == len(values_columns) == len(titles)):
        raise ValueError("All input lists must have the same length.")

    if category_orders is None:
        category_orders = [None] * len(dataframes)

    if len(category_orders) != len(dataframes):
        raise ValueError("category_orders must have the same length as dataframes or be None.")

    # Create the figure
    fig = go.Figure()

    # Add traces for each DataFrame
    for i, df in enumerate(dataframes):
        # Apply category order if provided
        if category_orders[i]:
            order = category_orders[i]
            df = df.copy()
            # Ensure the column is categorical with the correct categories and order
            categories = list(order.keys())
            df[labels_columns[i]] = pd.Categorical(df[labels_columns[i]], categories=categories, ordered=True)
            # Sort the DataFrame by the defined category order
            df.sort_values(labels_columns[i], inplace=True)

        pie = go.Pie(
            labels=df[labels_columns[i]],
            values=df[values_columns[i]],
            name=titles[i],
        )
        fig.add_trace(pie)

    # Generate buttons for toggling pie charts
    buttons = []
    for i, title in enumerate(titles):
        visibility = [False] * len(dataframes)
        visibility[i] = True  # Show only the current pie chart
        buttons.append({
            'label': title,
            'method': 'update',
            'args': [
                {'visible': visibility},
                {'title': title}  # Update the title dynamically
            ],
        })

    # Set initial layout with the first chart's title
    fig.update_layout(
        updatemenus=[{'buttons': buttons}],
        title=titles[0],  # Set the initial title
        width=width,  # Set figure width
        height=height  # Set figure height
    )

    # Set initial visibility
    for i, trace in enumerate(fig.data):
        trace.visible = (i == 0)  # Only the first chart is visible by default

    # Show the figure
    return fig

create_toggleable_pie_charts([active_years, not_active_years],
                                          ['Years bracket', 'Years bracket'],
                                          ['size', 'size'],
                                          ['Active Companies by Years Bracket', 'Not Active Companies by Years Bracket'],
                                   category_orders=[
    {"<1": 1, "1-5y": 2, "5-10y": 3, "10-20y": 4, ">20y": 5},
    {"<1": 1, "1-5y": 2, "5-10y": 3, "10-20y": 4, ">20y": 5}
])

In [19]:
def create_toggleable_pie_charts(dataframes, labels_columns, values_columns, titles, width=800, height=600):
    """
    Creates a Plotly figure with toggleable pie charts for multiple dataframes.

    Parameters:
        dataframes (list): List of DataFrames for pie charts.
        labels_columns (list): List of column names for labels in each DataFrame.
        values_columns (list): List of column names for values in each DataFrame.
        titles (list): List of titles for each pie chart.

    Returns:
        None: Displays the interactive chart.
    """
    if not (len(dataframes) == len(labels_columns) == len(values_columns) == len(titles)):
        raise ValueError("All input lists must have the same length.")

    # Create the figure
    fig = go.Figure()

    # Add traces for each DataFrame
    for i, df in enumerate(dataframes):
        pie = go.Pie(
            labels=df[labels_columns[i]],
            values=df[values_columns[i]],
            name=titles[i]
        )
        fig.add_trace(pie)

    # Generate buttons for toggling pie charts
    buttons = []
    for i, title in enumerate(titles):
        visibility = [False] * len(dataframes)
        visibility[i] = True  # Show only the current pie chart
        buttons.append({
            'label': title,
            'method': 'update',
            'args': [
                {'visible': visibility},
                {'title': title}  # Update the title dynamically
            ],
        })

    # Set initial layout with the first chart's title
    fig.update_layout(
        updatemenus=[{'buttons': buttons}],
        title=titles[0],  # Set the initial title
        width=width,  # Set figure width
        height=height  # Set figure height
    )

    # Set initial visibility
    for i, trace in enumerate(fig.data):
        trace.visible = (i == 0)  # Only the first chart is visible by default

    # Show the figure
    return fig
create_toggleable_pie_charts(
    [active_years, not_active_years],
    ['Years bracket', 'Years bracket'],
    ['size', 'size'],
    ['Active Companies by Years Bracket', 'Not Active Companies by Years Bracket'],
)